In [1]:
import sys
import os
import re
from tqdm import tqdm

sys.path.append("../LDP-RM")
from ldp_rm import LDP_RM
from data_rm import Data
from svsm_rm import SVSM
from metrics import Metrics

import ast

In [6]:
def analyze_cluster_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    limit = len(lines)
    domain_ids = set()

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if "(" in line:  # 格式是 like: (1, 2) # (3, 4)
            parts = line.split("#")
            for item in parts:
                try:
                    pair = ast.literal_eval(item.strip())
                    domain_ids.update(pair)
                except:
                    continue
        else:  # 格式是 like: 1 2 3 4 5
            ids = map(int, line.strip().split())
            domain_ids.update(ids)

    domain_size = max(domain_ids) + 1  # 假設 ID 從 0 開始

    return limit, domain_size, limit  # user_total = limit

def extract_sequence(line):
    pairs = re.findall(r"\((\d+),\s*(\d+)\)", line)
    sequence = [int(pairs[0][0])] + [int(p[1]) for p in pairs]
    return sequence

In [7]:
cluster_folder = "../dataset/cluster_movie/"
cluster_files = [
    f for f in os.listdir(cluster_folder) if re.match(r"cluster_\d+\.txt$", f)
]

# ===== 可調參數 =====
PRESET_TOP_K = 64  # 預設全域 top_k，可依需求自訂
TOP_KS = 1600  # 預設 top_ks
EPSILON = 4.0  # 預設 epsilon
SUBMAT = 4  # 預設 submat
# ====================

# filename = 'cluster_27.txt'
max_ncr = 0
max_f1 = 0
min_var = 1
ncr_sum = 0
f1_sum = 0
var_sum = 0
ct_sum = 0
total = 0

# global conf_dict
import heapq

top_rules_heap = []

In [12]:
max_file_ncr = ''
max_file_f1 = ''
for filename in sorted(cluster_files):
    # for filename in file:
    total += 1
    print(filename)
    cluster_path = filename
    cluster_path = os.path.join(cluster_folder, filename)
    cluster_id = filename.replace(".txt", "")
    # cluster_id = "movie_new2"

    # analyze cluster and it corresponding parameter for Data
    limit, domain_size, user_total = analyze_cluster_txt(cluster_path)

    if user_total < 10:
        continue

    print(limit, domain_size, user_total)
    # get the movie list in each cluster file
    with open(cluster_path, "r") as f:
        lines = f.readlines()
    cluster_size = len(lines)
    sequences = [
        extract_sequence(line) for line in tqdm(lines, desc="Extracting sequences...")
    ]
    movie_ids = sorted(set(movie_id for seq in sequences for movie_id in seq))
    if len(movie_ids) < 15:
        continue
    # 決定 epsilon
    epsilon = min(6.0, EPSILON * (400000 / cluster_size) ** 0.5)
    print(epsilon)
    # 根據每個 cluster 動態決定 top_k
    top_k = min(PRESET_TOP_K, int(len(movie_ids) * 0.5))
    # top_k = PRESET_TOP_K
    # 決定 top_ks
    top_ks = min(TOP_KS, int(top_k * (top_k - 1) / 2))  # 不超過可組合的 pair 數

    # 決定 top_kc
    top_kc = min(32, top_ks / 2)

    # Build Data, Metrics, LDP_RM parameters
    data = Data(
        dataname=cluster_id,
        limit=limit,
        domain_size=5020,
        user_total=user_total,
    )  # Movie dataset
    # metrics = Metrics(data, top_k=64, top_ks=1600, top_kc=32)
    metrics = Metrics(data, top_k=top_k, top_ks=top_ks, top_kc=top_kc)
    ldp_rm = LDP_RM(
        data, epsilon=epsilon, top_k=top_k, top_ks=top_ks, top_kc=top_kc, submat=SUBMAT
    )
    import time

    # 10 rounds
    for t in range(10):
        t1 = time.time()
        result_fre_dict_svd, result_conf_dict, hitrate_rm = ldp_rm.find_itemset_svd(
            task="RM",
            method="AMN",
            singnum=0.5,
            use_group=True,
            group_num=5,
            test="test_constant",
        )

        # 取 top 64
        for rule, confidence in result_conf_dict.items():
            entry = (confidence, rule)
            if len(top_rules_heap) < 32:
                heapq.heappush(top_rules_heap, entry)
            else:
                if confidence > top_rules_heap[0][0]:  # 比目前最小的還大
                    heapq.heappushpop(top_rules_heap, entry)
        final_conf_dict = dict(
            sorted(
                [(rule, confidence) for confidence, rule in top_rules_heap],
                key=lambda x: x[1],
                reverse=True,
            )
        )

        t2 = time.time()
        consume_time = int(t2 - t1)
        print("Final mining topks relations:", result_conf_dict)
        print("ldp_rm NCR", ncr := metrics.NCR(result_conf_dict))
        print("ldp_rm F1", f1 := metrics.F1(result_conf_dict))
        print("ldp_rm VAR", var := metrics.VARt(result_conf_dict))
        print("time:", ct := consume_time)
        ncr_sum += ncr
        f1_sum += f1
        var_sum += var
        ct_sum += ct
        max_ncr = max(max_ncr, ncr)
        max_f1 = max(max_f1, f1)
        if max_ncr == ncr:
            max_file_ncr = filename
        if max_f1 == f1:
            max_file_f1 = filename
        

cluster_0.txt
3274 5001 3274


Extracting sequences...: 100%|██████████| 3274/3274 [00:00<00:00, 219338.91it/s]


6.0
----------------------LDP-RM Result----------------------
***********SVIM Result**********
True top singleton of all users: {(1348,): 2542, (554,): 849, (2616,): 794, (551,): 724, (542,): 629, (1344,): 555, (2220,): 527, (164,): 523, (593,): 478, (159,): 457, (519,): 453, (169,): 452, (1585,): 430, (1920,): 419, (484,): 410, (2271,): 395, (197,): 393, (1294,): 389, (369,): 385, (49,): 373, (1347,): 373, (1368,): 363, (173,): 358, (1828,): 346, (341,): 333, (2574,): 319, (3017,): 315, (1209,): 304, (465,): 287, (2019,): 279, (1925,): 272, (2324,): 271, (187,): 264, (1352,): 264, (1574,): 262, (1973,): 254, (1619,): 251, (889,): 238, (1210,): 235, (3091,): 235, (1253,): 226, (1075,): 222, (1331,): 221, (6,): 219, (1795,): 219, (3535,): 218, (3092,): 214, (1343,): 209, (1257,): 206, (4968,): 203, (1227,): 202, (2388,): 197, (1378,): 192, (2680,): 189, (939,): 188, (1182,): 188, (296,): 188, (2000,): 186, (1192,): 185, (2325,): 183, (146,): 182, (899,): 180, (3886,): 177, (2968,): 175}

Extracting sequences...: 100%|██████████| 545/545 [00:00<00:00, 231225.54it/s]

6.0


...

6.0


----------------------LDP-RM Result----------------------
***********SVIM Result**********
True top singleton of all users: {(4881,): 3108, (1257,): 2068, (2798,): 2013, (1184,): 1644, (318,): 1343, (2775,): 1210, (846,): 1088, (2876,): 1044, (526,): 1007, (2489,): 900, (2774,): 893, (296,): 830, (260,): 825, (4901,): 817, (590,): 809, (355,): 763, (1172,): 755, (1187,): 687, (1176,): 675, (4135,): 672, (51,): 638, (593,): 603, (898,): 597, (899,): 585, (2679,): 556, (1174,): 553, (4794,): 552, (1198,): 521, (4215,): 514, (2247,): 511, (773,): 472, (1947,): 472, (3017,): 457, (111,): 452, (1175,): 450, (2687,): 447, (479,): 421, (49,): 415, (2807,): 412, (1828,): 404, (4903,): 404, (542,): 401, (1664,): 393, (740,): 384, (1245,): 381, (1567,): 370, (1128,): 364, (159,): 361, (344,): 355, (2680,): 355, (656,): 354, (2629,): 353, (2220,): 352, (889,): 351, (1235,): 350, (3490,): 350, (2616,): 345, (2242,): 341, (2546,): 337, (1182,): 334, (605,): 333, (1620,): 330, (2968,): 325, (896,): 

Extracting sequences...: 100%|██████████| 21714/21714 [00:00<00:00, 198383.13it/s]

6.0


----------------------LDP-RM Result----------------------
***********SVIM Result**********
True top singleton of all users: {(1795,): 2502, (582,): 1671, (1828,): 615, (1665,): 453, (146,): 443, (1925,): 427, (2687,): 424, (3017,): 422, (1920,): 419, (2271,): 395, (773,): 395, (32,): 395, (1344,): 394, (2325,): 390, (1548,): 376, (277,): 355, (344,): 353, (3886,): 345, (159,): 342, (1294,): 335, (1257,): 334, (4794,): 333, (45,): 325, (1347,): 320, (2968,): 312, (2019,): 311, (354,): 300, (1574,): 295, (1619,): 290, (2239,): 288, (656,): 285, (593,): 285, (1710,): 282, (1260,): 280, (1836,): 277, (1343,): 274, (1192,): 269, (1686,): 264, (1636,): 264, (542,): 264, (1620,): 261, (3535,): 254, (145,): 251, (775,): 246, (3171,): 243, (106,): 242, (2574,): 241, (256,): 240, (2247,): 236, (1073,): 233, (1837,): 228, (1348,): 227, (551,): 224, (2220,): 218, (49,): 216, (2000,): 215, (2616,): 213, (896,): 212, (3332,): 212, (715,): 205, (1973,): 203, (4215,): 203, (1069,): 202, (2388,): 201}


In [13]:
data = Data(
    dataname="movie_new2", limit=400000, domain_size=5020, user_total=400000
)  # Movie dataset
metrics = Metrics(data, top_k=64, top_ks=1600, top_kc=32)
print("===============")
print("# FINAL RESULT:")
print("Rules in final_conf_dict:", list(final_conf_dict.keys())[:64])
print("Rules in true_rules_dict:", list(metrics.true_rules_dict.keys())[:64])
print(
    "Intersected rules:",
    set(final_conf_dict).intersection(set(metrics.true_rules_dict)),
)

print("average NCR:", round(ncr_sum / total, 4))
print("average F1:", round(f1_sum / total, 4))
print("average consume time:", round(ct_sum / total, 4))
print("max NCR:", max_ncr)
print("max F1: ", max_f1)
print("Final mining topks relations:", final_conf_dict)
print("ldp_rm NCR", ncr := metrics.NCR(final_conf_dict))
print("ldp_rm F1", f1 := metrics.F1(final_conf_dict))

# FINAL RESULT:
Rules in final_conf_dict: [(4738, 3260), (1363, 1190), (1341, 161), (237, 1567), (701, 45), (3237, 4155), (1566, 3373), (1165, 1294), (542, 2296), (637, 1161), (4902, 3773), (2036, 920), (1377, 4902), (262, 1161), (920, 1837), (2069, 2616), (2271, 4465), (3189, 752), (605, 2186), (3903, 1378), (4012, 2212), (2212, 245), (3957, 3434), (260, 4992), (2616, 2263), (1238, 1161), (2618, 1795), (1176, 3373), (3899, 369), (2832, 113), (2969, 1999), (752, 3178), (1065, 113), (63, 3434), (63, 292), (3362, 3436), (1074, 2832), (484, 341), (49, 1070), (1347, 1795), (1006, 1837), (1006, 4902), (735, 538), (2271, 1348), (2907, 3037), (159, 346), (4662, 1962), (288, 3434), (288, 1910), (161, 231), (542, 1615), (3063, 1664), (3063, 846), (159, 574), (778, 792), (2629, 1836), (1587, 1377), (1837, 2535), (3826, 20), (3826, 3178), (3826, 1191), (4222, 20), (4222, 292), (4773, 102)]
Rules in true_rules_dict: [(5009, 5010), (5011, 5012), (5013, 5014), (5015, 5016), (5017, 5018), (5019, 5020

In [17]:
print("Rules in final_conf_dict:", list(final_conf_dict.keys())[:5])
print("Rules in true_rules_dict:", list(metrics.true_rules_dict.keys())[:5])

Rules in final_conf_dict: [(4738, 3260), (1363, 1190), (1341, 161), (237, 1567), (701, 45)]
Rules in true_rules_dict: [(5009, 5010), (5011, 5012), (5013, 5014), (5015, 5016), (5017, 5018)]


In [11]:
# 取前 32 筆（依 confidence 由大到小排序）
final_conf_dict_top32 = dict(
    sorted(final_conf_dict.items(), key=lambda x: x[1], reverse=True)[:32]
)
print("ldp_rm NCR", ncr := metrics.NCR(final_conf_dict_top32))
print("ldp_rm F1", f1 := metrics.F1(final_conf_dict_top32))

ldp_rm NCR 0.0
ldp_rm F1 0.0


In [14]:
print("max NCR:", max_ncr)
print("file: ", max_file_ncr)
print("max F1: ", max_f1)
print("file: ", max_file_f1)


max NCR: 0.7576
file:  cluster_36.txt
max F1:  0.7188
file:  cluster_36.txt
